In [1]:
from calitp.storage import get_fs
fs = get_fs()

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:81: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [2]:
from tqdm.notebook import tqdm
files = fs.expand_path('gs://test-calitp-gtfs-schedule-raw/schedule/', recursive=True)
files = [file for file in files if "ts" in file and fs.stat(file)["type"] != "directory"]
len(files)

3150

In [3]:
paths = [(path, *path.split("/")) for path in files]
paths[:5]

[('test-calitp-gtfs-schedule-raw/schedule/dt=1901-01-01/base64_url=aHR0cDovL21hcmludHJhbnNpdC5vcmcvZGF0YS9nb29nbGVfdHJhbnNpdC56aXA=/ts=1901-01-01T12:43:06.265686+00:00/valid_dir.zip',
  'test-calitp-gtfs-schedule-raw',
  'schedule',
  'dt=1901-01-01',
  'base64_url=aHR0cDovL21hcmludHJhbnNpdC5vcmcvZGF0YS9nb29nbGVfdHJhbnNpdC56aXA=',
  'ts=1901-01-01T12:43:06.265686+00:00',
  'valid_dir.zip'),
 ('test-calitp-gtfs-schedule-raw/schedule/dt=1901-01-01/base64_url=aHR0cDovL21hcmludHJhbnNpdC5vcmcvZGF0YS9nb29nbGVfdHJhbnNpdC56aXA=/ts=1901-01-01T13:43:06.265686+00:00/invalid_dir_toplevel.zip',
  'test-calitp-gtfs-schedule-raw',
  'schedule',
  'dt=1901-01-01',
  'base64_url=aHR0cDovL21hcmludHJhbnNpdC5vcmcvZGF0YS9nb29nbGVfdHJhbnNpdC56aXA=',
  'ts=1901-01-01T13:43:06.265686+00:00',
  'invalid_dir_toplevel.zip'),
 ('test-calitp-gtfs-schedule-raw/schedule/dt=1901-01-01/base64_url=aHR0cDovL21hcmludHJhbnNpdC5vcmcvZGF0YS9nb29nbGVfdHJhbnNpdC56aXA=/ts=1901-01-01T14:43:06.265686+00:00/invalid_dir_nested.zip

In [4]:
%pip install pendulum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.0/490.0 kB 19.9 MB/s eta 0:00:00
  Created wheel for pendulum: filename=pendulum-2.1.2-cp310-cp310-manylinux_2_35_x86_64.whl size=128338 sha256=a79cdc34e7dd7d2711bdb219819b3921151d57e845d708b6e9219c8789dcc6a5
  Stored in directory: /home/jovyan/.cache/pip/wheels/2e/41/ed/f113e4c9dc10f6c846d69f412e9cd9aa429960a3e2e485a4f9
Successfully built pendulum
Note: you may need to restart the kernel to use updated packages.


In [8]:
import pendulum
from datetime import datetime
moves = []
for og_path, bucket, table, dt, base64url, ts, filename in paths:
    pdt = pendulum.parse(dt.replace("dt=", ""), exact=True)
    assert isinstance(pdt, pendulum.Date)
    pts = pendulum.parse(time.replace("ts=", ""), exact=True)
    assert isinstance(pts, pendulum.DateTime)
    new_path = "/".join([bucket, table, dt, ts, base64url, filename])
    moves.append((og_path, new_path))
moves[:5]

[('test-calitp-gtfs-schedule-raw/schedule/dt=1901-01-01/base64_url=aHR0cDovL21hcmludHJhbnNpdC5vcmcvZGF0YS9nb29nbGVfdHJhbnNpdC56aXA=/ts=1901-01-01T12:43:06.265686+00:00/valid_dir.zip',
  'test-calitp-gtfs-schedule-raw/schedule/dt=1901-01-01/ts=1901-01-01T12:43:06.265686+00:00/base64_url=aHR0cDovL21hcmludHJhbnNpdC5vcmcvZGF0YS9nb29nbGVfdHJhbnNpdC56aXA=/valid_dir.zip'),
 ('test-calitp-gtfs-schedule-raw/schedule/dt=1901-01-01/base64_url=aHR0cDovL21hcmludHJhbnNpdC5vcmcvZGF0YS9nb29nbGVfdHJhbnNpdC56aXA=/ts=1901-01-01T13:43:06.265686+00:00/invalid_dir_toplevel.zip',
  'test-calitp-gtfs-schedule-raw/schedule/dt=1901-01-01/ts=1901-01-01T13:43:06.265686+00:00/base64_url=aHR0cDovL21hcmludHJhbnNpdC5vcmcvZGF0YS9nb29nbGVfdHJhbnNpdC56aXA=/invalid_dir_toplevel.zip'),
 ('test-calitp-gtfs-schedule-raw/schedule/dt=1901-01-01/base64_url=aHR0cDovL21hcmludHJhbnNpdC5vcmcvZGF0YS9nb29nbGVfdHJhbnNpdC56aXA=/ts=1901-01-01T14:43:06.265686+00:00/invalid_dir_nested.zip',
  'test-calitp-gtfs-schedule-raw/schedule/dt=19

In [ ]:
for og_path, new_path in tqdm(moves):
    #print(og_path, new_path)
    #break
    fs.mv(og_path, new_path)